In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
from sentence_transformers import SentenceTransformer,  models, util

In [3]:
trained_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [4]:
movies = pd.read_table("datasets/movie_datasets/imdb/title.basics.tsv", sep="\t")

C:\Users\smend\AppData\Local\Temp\ipykernel_27804\3062502678.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_table("datasets/movie_datasets/imdb/title.basics.tsv", sep="\t")


In [5]:
movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [6]:
movies = movies[movies["titleType"]=="movie"]

In [7]:
movies[movies.primaryTitle.str.lower().str.contains("superman", na=False)]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
40141,tt0040852,movie,Superman,Superman,0,1948,\N,244,Sci-Fi
41472,tt0042211,movie,Atom Man vs. Superman,Atom Man vs. Superman,0,1950,\N,252,"Family,Sci-Fi"
43315,tt0044091,movie,Superman and the Mole-Men,Superman and the Mole-Men,0,1951,\N,58,"Action,Adventure,Drama"
46688,tt0047545,movie,Superman Flies Again,Superman Flies Again,0,1954,\N,77,"Action,Fantasy,Sci-Fi"
46689,tt0047546,movie,Superman and the Jungle Devil,Superman and the Jungle Devil,0,1954,\N,77,"Action,Fantasy,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
8954890,tt6992282,movie,Superman vs. The KKK,Superman vs. The KKK,0,\N,\N,\N,Thriller
9176903,tt7480652,movie,Secret Identity: The Origin of Superman,Secret Identity: The Origin of Superman,0,\N,\N,\N,Documentary
9218495,tt7571716,movie,Superman The Power,Superman The Power,0,2018,\N,\N,Action
9501476,tt8188984,movie,My Son Superman,My Son Superman,0,2016,\N,45,Comedy


In [8]:
movies_dict = {}
movies_list = []

In [9]:
for i, row in tqdm(movies[movies["titleType"]=="movie"].iterrows()):
    movies_dict[row["originalTitle"]]=row["tconst"]
    movies_list.append(row["originalTitle"])

662293it [01:01, 10856.27it/s]


In [10]:
def generate_embeddings(string):
    return trained_model.encode(string, convert_to_tensor=True, normalize_embeddings=True, show_progress_bar=True)

In [11]:
movie_embeddings = generate_embeddings(movies_list)

Batches:   0%|          | 0/20697 [00:00<?, ?it/s]

In [12]:
def retreive_top_k_results(question, topk=1):
    question_embeddings = generate_embeddings(question)
    result =  util.semantic_search(question_embeddings, movie_embeddings, top_k=topk)[0][0]
    return  result["score"], result["corpus_id"]

In [13]:
question = "Who were the stars in the movie superman the power?"

In [14]:
score, id = retreive_top_k_results(question)
print(movies_list[id], movies_dict[movies_list[id]], score)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Superman The Power tt7571716 0.7681993246078491


'tt0848228'